# Clustering 

In [1]:
# Load Libraries
import pandas as pd
import re
import warnings

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import RFECV

from sklearn.metrics import mean_squared_error, r2_score, confusion_matrix, accuracy_score, f1_score, precision_score, recall_score
from math import sqrt
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge


from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import LabelEncoder

import matplotlib.pyplot as plt

# machine learning
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
import itertools
import matplotlib.pyplot as plt
import numpy as np

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()
    plt.show()

In [3]:
#Load the data

df_movies = pd.read_csv("regressionPreprocessing.csv")
print("Length after import: " + str(len(df_movies)))
df_movies = df_movies.fillna(0)
df_movies.head(5)

Length after import: 7517


budget    id  runtime  vote_count  History  Western  Music  Family  \
0  30000000   862     81.0        5415      0.0      0.0    0.0     1.0   
1  65000000  8844    104.0        2413      0.0      0.0    0.0     1.0   
2  60000000   949    170.0        1886      0.0      0.0    0.0     0.0   
3  58000000   710    130.0        1194      0.0      0.0    0.0     0.0   
4  98000000  1408    119.0         137      0.0      0.0    0.0     0.0   

   Comedy  Drama  ...  orig_ml  orig_es  orig_bn  orig_ab  orig_wo  orig_ca  \
0     1.0    0.0  ...      0.0      0.0      0.0      0.0      0.0      0.0   
1     0.0    0.0  ...      0.0      0.0      0.0      0.0      0.0      0.0   
2     0.0    1.0  ...      0.0      0.0      0.0      0.0      0.0      0.0   
3     0.0    0.0  ...      0.0      0.0      0.0      0.0      0.0      0.0   
4     0.0    0.0  ...      0.0      0.0      0.0      0.0      0.0      0.0   

   orig_ru  orig_id  orig_is  director  
0      0.0      0.0      0.0      1709  
1      0.0      0.0      0.0      1645  
2      0.0      0.0      0.0      2338  
3      0.0      0.0      0.0      2231  
4      0.0      0.0      0.0      2812  

[5 rows x 90 columns]

In [4]:
# drop columns that are not needed
features_to_remove = ['vote_count','+18','id','actors','spokenLanguages','productionCountries','productionCompanies','original_language','hasHomepage','part_of_collection''orig_lv', 'orig_el', 'orig_fa', 'orig_tl', 'orig_ta', 'orig_th',
       'orig_mn', 'orig_zh', 'orig_te', 'orig_kk', 'orig_zu', 'orig_et',
       'orig_mr', 'orig_eu', 'orig_sv', 'orig_no', 'orig_pl', 'orig_cs',
       'orig_cy', 'orig_bs', 'orig_de', 'orig_lo', 'orig_xx', 'orig_ko',
       'orig_hu', 'orig_sr', 'orig_da', 'orig_pt', 'orig_nl', 'orig_en',
       'orig_it', 'orig_tr', 'orig_hr', 'orig_cn', 'orig_ka', 'orig_ar',
       'orig_ja', 'orig_0', 'orig_hi', 'orig_ro', 'orig_af', 'orig_sk',
       'orig_fr', 'orig_fi', 'orig_he', 'orig_uk', 'orig_bg', 'orig_ml',
       'orig_es', 'orig_bn', 'orig_ab', 'orig_wo', 'orig_ca', 'orig_ru',
       'orig_id', 'orig_is', 'orig_lv',
        'runtime',
       'part_of_collection', '18+']
for i in features_to_remove:
    if i in df_movies.columns:
        df_movies = df_movies.drop(columns=i)
print(df_movies.head(5))
df_movies.columns

     budget  History  Western  Music  Family  Comedy  Drama  Foreign  Action  \
0  30000000      0.0      0.0    0.0     1.0     1.0    0.0      0.0     0.0   
1  65000000      0.0      0.0    0.0     1.0     0.0    0.0      0.0     0.0   
2  60000000      0.0      0.0    0.0     0.0     0.0    1.0      0.0     1.0   
3  58000000      0.0      0.0    0.0     0.0     0.0    0.0      0.0     1.0   
4  98000000      0.0      0.0    0.0     0.0     0.0    0.0      0.0     1.0   

   Horror  ...  Documentary  Thriller  TV Movie  Crime  Science Fiction  \
0     0.0  ...          0.0       0.0       0.0    0.0              0.0   
1     0.0  ...          0.0       0.0       0.0    0.0              0.0   
2     0.0  ...          0.0       1.0       0.0    1.0              0.0   
3     0.0  ...          0.0       1.0       0.0    0.0              0.0   
4     0.0  ...          0.0       0.0       0.0    0.0              0.0   

   Fantasy  War  Adventure    rating  director  
0      0.0  0.0    

Index(['budget', 'History', 'Western', 'Music', 'Family', 'Comedy', 'Drama',
       'Foreign', 'Action', 'Horror', 'Mystery', 'Romance', 'Animation',
       'Documentary', 'Thriller', 'TV Movie', 'Crime', 'Science Fiction',
       'Fantasy', 'War', 'Adventure', 'rating', 'director'],
      dtype='object')

In [5]:
# separate features and target variable
rating = df_movies['rating'] # weight
features = df_movies.drop(columns=['rating'])

# encode labels
lab_enc = LabelEncoder()
rating = lab_enc.fit_transform(rating)

# create a train/test split
features_train, features_test, rating_train, rating_test = train_test_split(
    features, rating, test_size=0.4, random_state=42)

print("Train: " + str(len(features_train)) + " Features and " + str(len(rating_train)) + " Ratings")
print("Test: " + str(len(features_test)) + " Features and " + str(len(rating_test)) + " Ratings")



Train: 4510 Features and 4510 Ratings
Test: 3007 Features and 3007 Ratings


In [6]:
# create and fit a knn classifier
knn_reg = KNeighborsClassifier(n_neighbors=1000)
knn_reg.fit(features_train, rating_train)
rating_pred_knn = knn_reg.predict(features_test)

print("Scores knn:")
#compute the confusion matrix
cnf_matrix = confusion_matrix(rating_test, rating_pred_knn)
print(cnf_matrix)

#compute accuracy score
print("Accuracy: {}".format(accuracy_score(rating_test, rating_pred_knn)))
print("Precision: {}".format(precision_score(rating_test, rating_pred_knn, average='weighted')))
print("Recall: {}".format(recall_score(rating_test, rating_pred_knn, average='weighted')))
print("f1_score: {}".format(f1_score(rating_test, rating_pred_knn, average='weighted')))


#plot the confusion matrix
#plot_confusion_matrix(cnf_matrix, classes=lab_enc.classes_, title='KNN Classifier')

print()

Scores knn:
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
Accuracy: 0.035583638177585634
Precision: 0.0024860149416241054
Recall: 0.035583638177585634
f1_score: 0.0045887389910926404



C:\Users\d060445\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\d060445\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [7]:
##### create and fit a RandomForestClassifier
rf_reg = RandomForestClassifier()
rf_reg.fit(features_train, rating_train)
rating_pred_rf = rf_reg.predict(features_test)

print("Scores knn:")
#compute the confusion matrix
cnf_matrix = confusion_matrix(rating_test, rating_pred_rf)
print(cnf_matrix)

#compute accuracy score
print("Accuracy: {}".format(accuracy_score(rating_test, rating_pred_rf)))
print("Precision: {}".format(precision_score(rating_test, rating_pred_rf, average='weighted')))
print("Recall: {}".format(recall_score(rating_test, rating_pred_rf, average='weighted')))
print("f1_score: {}".format(f1_score(rating_test, rating_pred_rf, average='weighted')))

#plot the confusion matrix
#plot_confusion_matrix(cnf_matrix, classes=lab_enc.classes_, title='KNN Classifier')

print()


Scores knn:
[[1 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
Accuracy: 0.008313934153641503
Precision: 0.007210381211635159
Recall: 0.008313934153641503
f1_score: 0.007235894096123855



C:\Users\d060445\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\d060445\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\d060445\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\d060445\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [8]:
##### create and fit a DecisionTreeClassifier
dt_reg = DecisionTreeClassifier()
dt_reg.fit(features_train, rating_train)
rating_pred_dt = dt_reg.predict(features_test)

print("Scores knn:")
#compute the confusion matrix
cnf_matrix = confusion_matrix(rating_test, rating_pred_dt)
print(cnf_matrix)

#compute accuracy score
print("Accuracy: {}".format(accuracy_score(rating_test, rating_pred_dt)))
print("Precision: {}".format(precision_score(rating_test, rating_pred_dt, average='weighted')))
print("Recall: {}".format(recall_score(rating_test, rating_pred_dt, average='weighted')))
print("f1_score: {}".format(f1_score(rating_test, rating_pred_dt, average='weighted')))

#plot the confusion matrix
#plot_confusion_matrix(cnf_matrix, classes=lab_enc.classes_, title='KNN Classifier')

print()
#from sklearn import tree
#tree.plot_tree(dt_reg)

Scores knn:
[[1 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [1 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
Accuracy: 0.008979048885932824
Precision: 0.007415882136468402
Recall: 0.008979048885932824
f1_score: 0.007582853072449445



C:\Users\d060445\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\d060445\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\d060445\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\d060445\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [9]:
##### create and fit a GaussianNB
nb_reg = GaussianNB()
nb_reg.fit(features_train, rating_train)
rating_pred_nb = nb_reg.predict(features_test)

print("Scores knn:")
#compute the confusion matrix
cnf_matrix = confusion_matrix(rating_test, rating_pred_nb)
print(cnf_matrix)

#compute accuracy score
print("Accuracy: {}".format(accuracy_score(rating_test, rating_pred_nb)))
print("Precision: {}".format(precision_score(rating_test, rating_pred_nb, average='weighted')))
print("Recall: {}".format(recall_score(rating_test, rating_pred_nb, average='weighted')))
print("f1_score: {}".format(f1_score(rating_test, rating_pred_nb, average='weighted')))


#plot the confusion matrix
#plot_confusion_matrix(cnf_matrix, classes=lab_enc.classes_, title='KNN Classifier')

print()


Scores knn:
[[6 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 ...
 [1 0 0 ... 0 0 0]
 [8 0 0 ... 0 0 0]
 [5 0 0 ... 0 0 0]]
Accuracy: 0.0029930162953109413
Precision: 0.005010749114532075
Recall: 0.0029930162953109413
f1_score: 0.0016524254020128335



C:\Users\d060445\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\d060445\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\d060445\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\d060445\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [10]:
##### create and fit a SVC
svc_reg = SVC()
svc_reg.fit(features_train, rating_train)
rating_pred_svc = svc_reg.predict(features_test)

print("Scores knn:")
#compute the confusion matrix
cnf_matrix = confusion_matrix(rating_test, rating_pred_svc)
print(cnf_matrix)

#compute accuracy score
print("Accuracy: {}".format(accuracy_score(rating_test, rating_pred_svc)))
print("Precision: {}".format(precision_score(rating_test, rating_pred_svc, average='weighted')))
print("Recall: {}".format(recall_score(rating_test, rating_pred_svc, average='weighted')))
print("f1_score: {}".format(f1_score(rating_test, rating_pred_svc, average='weighted')))

#plot the confusion matrix
#plot_confusion_matrix(cnf_matrix, classes=lab_enc.classes_, title='KNN Classifier')

print()


Scores knn:
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
Accuracy: 0.03392085134685733
Precision: 0.005814761747761968
Recall: 0.03392085134685733
f1_score: 0.0067314507175342524



C:\Users\d060445\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\d060445\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\d060445\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\d060445\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
